<a href="https://colab.research.google.com/github/rjaa2020/projects/blob/main/blackjack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

In [7]:
class Deck():
    def __init__(self, n_decks=1):
        self.values = ['Ace', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Jack', 'Queen', 'King']
        self.suites = ['Spades', 'Hearts', 'Clubs', 'Diamonds']
        self.deck = [f"{value} of {suite}" for suite in self.suites for value in self.values] * n_decks
        self.deck_values = {f"{value} of {suite}": (10 if isinstance(value, str) and value != 'Ace' else value) for suite in self.suites for value in self.values}
        self.deck_size = len(self.deck)

    def update_deck_size(self):
        self.deck_size = len(self.deck)

class Player():
    def __init__(self, name):
        self.hand = []
        self.hand_value = 0
        self.name = name
        self.bust = False

    def check_bust(self):
        if self.hand_value > 21:
            self.bust = True
            print(f'------{self.name} busted!------')

    def __repr__(self):
        return self.name

class BlackJack():
    def __init__(self, n_players=1, n_decks=1, seed=None):
        assert n_players > 0, "Must be at least one player"
        assert n_decks > 0, "Must be at least one"

        if seed is not None:
            random.seed(seed)

        self.deck = Deck(n_decks)
        self.dealer = Player("Dealer")
        self.players = [Player(f"Player {i}") for i in range(1, n_players + 1)]
        self.players_left = n_players

    def get_card_value(self, card, player):
        if isinstance(card, str):
            if 'Ace' in card:
                return 11 if player.hand_value + 11 <= 21 else 1
        return self.deck.deck_values[card]

    def pick_one_card(self, player):
        if player.bust:
            print(f"{player} cannot draw any more cards")
        else:
            card = self.deck.deck.pop(random.randint(0, len(self.deck.deck) - 1))
            self.deck.update_deck_size()
            value = self.get_card_value(card, player)

            player.hand += [card]
            player.hand_value += value
            player.check_bust()

            return card

    def display_board(self):
        print("------------------------------")
        print(f"Dealer's hand: [{self.dealer.hand[0]}, ?]")
        for player in self.players:
            print(f"{player}'s hand: {player.hand}")

    def play_game(self):
        for _ in range(2):
            for player in [self.dealer] + self.players:
                self.pick_one_card(player)
        self.display_board()

        for player in self.players:
            while not player.bust:
                print("------------------------------")
                print(f"{player}'s turn:")
                if input(f"Current value is {player.hand_value}. Do you want to hit? (yes/no): ").lower() == 'yes':
                    self.pick_one_card(player)
                    self.display_board()
                else:
                    break

        # Dealer's turn
        print("------------------------------")
        print("Dealer's turn:")
        print(f"{self.dealer}'s hand: [{self.dealer.hand[0]}, ?]")
        self.display_board()
        while not self.dealer.bust and self.dealer.hand_value < 17:
            self.pick_one_card(self.dealer)
            self.display_board()

        # Determine winners
        print("------------------------------")
        print("Final state of the game:")
        print(f"{self.dealer}'s hand: {self.dealer.hand}")
        for player in self.players:
            if not player.bust:
                if self.dealer.bust or player.hand_value > self.dealer.hand_value:
                    print(f"{player} wins!")
                elif player.hand_value < self.dealer.hand_value:
                    print(f"{player} loses!")
                else:
                    print(f"{player} ties with the dealer!")

In [8]:
blackjack_game = BlackJack(n_players=1, n_decks=1)
blackjack_game.play_game()

------------------------------
Dealer's hand: [9 of Diamonds, ?]
Player 1's hand: ['9 of Hearts', 'Queen of Clubs']
------------------------------
Player 1's turn:
Current value is 19. Do you want to hit? (yes/no): no
------------------------------
Dealer's turn:
Dealer's hand: [9 of Diamonds, ?]
------------------------------
Dealer's hand: [9 of Diamonds, ?]
Player 1's hand: ['9 of Hearts', 'Queen of Clubs']
------------------------------
Final state of the game:
Dealer's hand: ['9 of Diamonds', '8 of Hearts']
Player 1 wins!
